```
This software is a part of GPU Ocean.

Copyright (C) 2019  SINTEF Digital

This notebook is a prototype for all required parts of a realistic simulation

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset

#Helpful during development
import importlib
#importlib.reload(module)

import datetime

from IPython.display import display

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc


#Set large figure sizes
#plt.rcParams["animation.html"] = "jshtml" #Javascript 
plt.rcParams["animation.html"] = "html5" #mp4
plt.rcParams["figure.dpi"] = 100.0 #highres movies/plots

plt.rcParams["animation.writer"] = 'ffmpeg' 
plt.rcParams["animation.codec"] = 'h264' 

plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = 'serif'

In [2]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

from SWESimulators import CDKLM16, Common, IPythonMagic, NetCDFInitialization
from SWESimulators import GPUDrifterCollection, Observation, ParticleInfo
from SWESimulators import DataAssimilationUtils as dautils

from demos.realisticSimulations import norkyst_plotting


In [3]:
%cuda_context_handler gpu_ctx

# Read observation file and plot some results

In [7]:
from SWESimulators import SimReader
#observation_file = "2019-12-02/lovese/observation_10_05_44.pickle"
#truth_reader = SimReader.SimNetCDFReader("2019-12-02/lovese/lovese_original.nc")

#observation_file = "2019-12-03/lovese/opendrift_observations_18_08_56.pickle"
#truth_reader = SimReader.SimNetCDFReader("2019-11-28/lovese/lovese_original.nc")

observation_file = "2019-12-03/lovese/opendrift_observations_15_06_29.pickle"



### Using interpolation factor 7
#result_folder = 'EPS_2019_11_30-19_35_00/' # Example using    default q0: 8e-3
#result_folder = 'EPS_2019_11_30-19_09_17/' # Example using very small q0: 1e-5
#result_folder = 'EPS_2019_11_30-19_38_52/' # Example using      small q0: 1e-4
#result_folder = 'EPS_2019_11_30-19_50_45/' # Example using      large q0: 1e-3
#result_folder = 'EPS_2019_11_30-19_58_41/' # Example using     medium q0: 5e-4
#result_folder = 'EPS_2019_11_30-20_15_17/' # Example using      small q0: 0.7e-4
#result_folder = 'EPS_2019_11_30-20_18_59/' # Example usingquite small q0: 0.5e-4


### Using interpolation factor 9
#result_folder = 'EPS_2019_11_30-20_02_26/' # Example using     medium q0: 5e-4
#result_folder = 'EPS_2019_11_30-20_06_38/' # Example using      small q0: 1e-4

### Using interpolation factor 5
#result_folder = 'EPS_2019_11_30-20_09_39/' # Example using      small q0: 1e-4
#result_folder = 'EPS_2019_11_30-20_12_57/' # Example using      small q0: 0.7e-4

# With random q0 and random cov-radius
#result_folder = 'EPS_2019_11_30-20_22_57/' # Example q0 [0.5e-4 - 7.5e-4]
#result_folder = 'EPS_2019_11_30-20_29_16/' # Example q0 [0.5e-4 - 5e-4]
#result_folder = 'EPS_2019_11_30-20_40_59/' # Example q0 [0.3e-4 - 3e-4]


### With DA
#result_folder = 'EPS_2019_11_30-20_46_41/' # Example q0 [0.5e-4 - 5e-4]
#result_folder = 'EPS_2019_11_30-20_57_37/' # Example q0 [0.5e-4 - 5e-4]

### DGX-2 results
#result_folder = 'EPS_2019_12_03-14_25_42/'
#result_folder = 'EPS_2019_12_03-19_36_33/'

# mpirun -n 16 python3 sequential_importance_resampling.py --per_node_ensemble_size 63 --observation_file 2019-12-03/lovese/opendrift_observations_15_06_29.pickle --observation_variance 10 --resampling_frequency_minutes 30 --data_assimilation_end_hours 48 --forecast_duration_hours 47
result_folder = 'EPS_2019_12_04-16_44_55/'

# mpirun --mca btl_openib_if_include mlx5_0 --mca btl_openib_warn_no_device_params_found 0 /home/martinls/miniconda3/envs/gpuocean/bin/python3 sequential_importance_resampling.py --per_node_ensemble_size 21 --observation_file 2019-12-03/lovese/opendrift_observations_15_06_29.pickle --observation_variance 10 --resampling_frequency_minutes 5 --data_assimilation_end_hours 48 --forecast_duration_hours 47
#result_folder = 'EPS_30821_2019_12_05-21_42_22/'

### Deterministic data
deterministic_filename = os.path.join('deterministic_forecast_2days_2019_12_06-10_09_45', 'trajectory_forecast_48_to_95.pickle')
#deterministic_filename = os.path.join('deterministic_forecast_12hours_2019_12_06-09_59_08', 'trajectory_forecast_12_to_24.pickle')


eta = truth_reader.getLastTimeStep()[0]

obs = Observation.Observation(observation_type=dautils.ObservationType.UnderlyingFlow,
                              domain_size_x=truth_reader.get("nx")*truth_reader.get("dx"), domain_size_y=truth_reader.get("ny")*truth_reader.get("dy"),
                              nx=truth_reader.get("nx"), ny=truth_reader.get("ny"),
                              land_mask=eta.mask)
obs.read_pickle(observation_file)

Exception caught: Resetting to CUDA context gpu_ctx
Traceback (most recent call last):
  File "C:\Users\havardh\AppData\Local\Continuum\anaconda3\envs\gpuocean_py36\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-07be8deefd16>", line 55, in <module>
    eta = truth_reader.getLastTimeStep()[0]
NameError: name 'truth_reader' is not defined


NameError: name 'truth_reader' is not defined

In [ ]:
analyse_particle_infos = False

particle_info_dir = os.path.join(result_folder, "particle_info")
filenames = os.listdir(particle_info_dir)

particle_infos = [None] * len(filenames)

if analyse_particle_infos:
    for i in range(len(particle_infos)):
        particle_infos[i] = ParticleInfo.ParticleInfo()
        particle_infos[i].read_pickle(os.path.join(particle_info_dir, filenames[i]))

In [ ]:
extra_cell_id = 19


def state_at_recorded_cell(extra_cell_id):

    def get_time_series_for_particle(particle_info, extra_cell_id):

        time  = particle_info.get_sample_times()
        hu = np.empty_like(time)
        hv = np.empty_like(time)

        all_buoy_obs = particle_info.state_df["extra_states"]
        for t in range(len(time)):
            hu[t] = all_buoy_obs[t][extra_cell_id,1]
            hv[t] = all_buoy_obs[t][extra_cell_id,2]

        return time, hu, hv

    #obs_t, obs_hu, obs_hv = get_time_series_for_buoy(obs, buoy_id)

    fig = plt.figure(figsize=(8,4))
    plt.legend()
    plt.title('')

    t, _, _ = get_time_series_for_particle(particle_infos[0], extra_cell_id)
    hu_mean = np.zeros_like(t)
    for particle_info in particle_infos:
        t, hu, _ = get_time_series_for_particle(particle_info, extra_cell_id)
        hu_mean += hu
        plt.plot(t, hu, color='xkcd:light blue grey', alpha=0.6)

    hu_mean = hu_mean/len(particle_infos)
    plt.plot(t, hu_mean, color="red", alpha=0.6)
    #plt.plot(obs_t[:25], obs_hu[:25])
    plt.title("hu")

    fig = plt.figure(figsize=(8,4))
    plt.legend()
    plt.title('')

    hv_mean = np.zeros_like(t)
    for particle_info in particle_infos:
        t, _, hv = get_time_series_for_particle(particle_info, extra_cell_id)
        hv_mean += hv
        plt.plot(t, hv, color='xkcd:light blue grey', alpha=0.6)


    hv_mean = hv_mean/len(particle_infos)
    plt.plot(t, hv_mean, color="red", alpha=0.6)
    #plt.plot(obs_t[:25], obs_hv[:25])
    plt.title("hv")

    #plt.xlim([800, 1000])

    print(t.shape)
    print(type(hu), hu.shape)

if analyse_particle_infos:
    state_at_recorded_cell(extra_cell_id)
else:
    print('We skip this part...')

# Plot drifter trajectory forecast

In [ ]:
drifter_forecast_dir = os.path.join(result_folder, "forecast_observation_files")
filenames = os.listdir(drifter_forecast_dir)

drifter_forecast = [None] * len(filenames)
for i in range(len(drifter_forecast)):
    drifter_forecast[i] = Observation.Observation(domain_size_x=truth_reader.get("nx")*truth_reader.get("dx"), domain_size_y=truth_reader.get("ny")*truth_reader.get("dy"),
                              nx=truth_reader.get("nx"), ny=truth_reader.get("ny"),
                              land_mask=eta.mask)
    drifter_forecast[i].read_pickle(os.path.join(drifter_forecast_dir,filenames[i]))
    
deterministic_drifter_forecast = Observation.Observation(domain_size_x=truth_reader.get("nx")*truth_reader.get("dx"), domain_size_y=truth_reader.get("ny")*truth_reader.get("dy"),
                              nx=truth_reader.get("nx"), ny=truth_reader.get("ny"),
                              land_mask=eta.mask)
deterministic_drifter_forecast.read_pickle(deterministic_filename)

In [ ]:
#drifter_id = 41
#drifter_id = 84
#drifter_id = 85
drifter_id = 103


start_time_forecast = 60*60
end_time_forecast = 2*60*60

#start_time_forecast = 48*60*60
#end_time_forecast = 95*60*60




drifter_paths = [None]*len(drifter_forecast)
for i in range(len(drifter_paths)):
    drifter_paths[i] = drifter_forecast[i].get_drifter_path(drifter_id, start_time_forecast, end_time_forecast)

deterministic_drifter_path = deterministic_drifter_forecast.get_drifter_path(drifter_id, start_time_forecast, end_time_forecast)
    
truth_paths = obs.get_drifter_path(drifter_id, start_time_forecast, end_time_forecast)

mean_path = drifter_forecast[0].get_drifter_path(drifter_id,  
                                                 start_time_forecast,
                                                 end_time_forecast,
                                                 keepDomainSize=False)[0]
num_paths_in_mean = 1
            
for i in range(1, len(drifter_forecast)):
    tmp_mean_path = drifter_forecast[i].get_drifter_path(drifter_id,  
                                                         start_time_forecast,
                                                         end_time_forecast,
                                                         keepDomainSize=False)[0]
    if not np.isnan(tmp_mean_path).any():
        mean_path += tmp_mean_path
        num_paths_in_mean += 1
                    
mean_path /= num_paths_in_mean

In [ ]:
def forecastPlot(ax, true_paths, ensemble_paths, deterministic_paths=None, color_id=0, 
                 forecast_point_size=2, truth_point_size=6, mean_paths=[None]):
    """
    ax: Axis object to draw trajectories in
    true_path: A single list of paths that represents the truth
    ensemble_paths: Multiple lists of paths that represents the forecast
    color_id: integer representing a pre-defined color combination.
    """

    color_combinations = [
        # [true color, forecast color]
        #['xkcd:dark grey blue', 'xkcd:light blue grey', 'xkcd:wine'],
        ['xkcd:scarlet', 'xkcd:light blue grey', 'xkcd:dark grey blue'],
        ['xkcd:viridian', 'xkcd:foam green']
    ]
    
    assert(color_id < len(color_combinations)), 'Invalid color_id'
    
    true_color = color_combinations[color_id][0]
    forecast_color = color_combinations[color_id][1]
    mean_color = color_combinations[color_id][2]

    # Plot forecast paths 
    for paths in ensemble_paths:
        for path in paths:
            ax.plot(path[:,0], path[:,1], color=forecast_color, alpha=0.1)
        
        # Mark end position of forecast
        end_position   = paths[-1][-1,:]
        circ_end_forecast = matplotlib.patches.Circle((end_position[0], end_position[1]), 
                                                      forecast_point_size, 
                                                      fill=False, zorder=10,
                                                      alpha=0.7)
        ax.add_patch(circ_end_forecast)

    
    # Plot true path
    for path in true_paths:
        ax.plot(path[:,0], path[:,1], color=true_color, zorder=5)
        
    # Plot deterministic path
    if deterministic_paths is not None:
        for path in deterministic_paths:
            ax.plot(path[:,0], path[:,1], color='g', zorder=7)
    
    # Plot mean path
    if mean_paths[0] is not None:
        for path in mean_paths:
            ax.plot(path[:,0], path[:,1], color=mean_color, zorder=6)
    
    # Mark start and end of true path
    start_pos = true_paths[0][0,:]
    end_pos   = true_paths[-1][-1,:]
    circ_start = matplotlib.patches.Circle((start_pos[0], start_pos[1]), 
                                           truth_point_size,
                                           fill=False, zorder=10)
    ax.add_patch(circ_start)
    #circ_end = matplotlib.patches.Circle((end_pos[0], end_pos[1]), 
    #                                     truth_point_size,
    #                                     fill=False, zorder=10)
    #ax.add_patch(circ_end)
    ax.plot(end_pos[0], end_pos[1], 'x', color='k', zorder=11)
    
    if deterministic_paths is not None:
        ax.plot(deterministic_paths[-1][-1,0], deterministic_paths[-1][-1,1], 
                   marker='o', markeredgecolor='g', markerfacecolor=None)
    
    # Mark end of mean path
    if mean_paths[0] is not None:
        #xend = matplotlib.patches.Circle((mean_paths[-1][-1,0], mean_paths[-1][-1,1]),
        #                                 truth_point_size,
        #                                 fill=False, zorder=10)
        #ax.add_patch(xend)
        ax.plot(mean_paths[-1][-1,0], mean_paths[-1][-1,1], 
                   marker='o', markeredgecolor='k', markerfacecolor='b')
        
def createForecastCanvas(observation):
    """
    true_sim_reader: The SimReader object containing nx, ny, etc
    """   
    
    nx = observation.nx
    ny = observation.ny
    
    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(111)
    
    extent=np.array([0, observation.domain_size_x, 0, observation.domain_size_y])/1000
    ax.imshow(observation.land_mask, origin="lower", 
              extent=extent, cmap='binary')

    return ax



#ax = createForecastCanvas(obs)
fig, axs = plt.subplots(1, 1, figsize=(7,7))

print(len(drifter_paths[0]))
print(drifter_paths[0][0].shape)
forecastPlot(axs, truth_paths, drifter_paths, deterministic_drifter_path, forecast_point_size=0.002, truth_point_size=0.001, mean_paths=[mean_path])

fig.savefig("drifter_" + str(drifter_id) + ".png")


In [ ]:
print(len(deterministic_drifter_path))
deterministic_drifter_path[0].shape

In [ ]:
for i in range(10):
    print(np.random.rand()*10)

In [ ]:
hpc_joule_per_sec = 10096*1000 
print(hpc_joule_per_sec)
print((hpc_joule_per_sec*(3600)/1000))


In [ ]:
oslo_household_kWh = 12600
oslo_household_joule_per_year = oslo_household_kWh*1000*3600
print(oslo_household_joule_per_year)
print(oslo_household_joule_per_year/hpc_joule_per_sec)
